In [1]:
from gmath import *
from process import *

[Taichi] version 1.6.0, llvm 15.0.1, commit f1c6fbbd, win, python 3.9.13
[Taichi] Starting on arch=cuda


In [2]:
pixels, c2w=load('HotDog')
rays = ti.Vector.field(3,ti.f32, shape=(len(pixels), pixels[0].shape[0], pixels[0].shape[1]))
origins = ti.Vector.field(3,ti.f32, shape=(len(pixels)))
print(len(pixels),c2w.shape,rays.shape,origins.shape)

Loading HotDog/rgb/


 25%|██▌       | 100/400 [00:11<00:35,  8.37it/s]


100 images loaded successfully with the shape of (800, 800)
Loading HotDog/pose/


 25%|██▌       | 100/400 [00:00<00:00, 2940.89it/s]


100 matrices loaded successfully
100 (100,) (100, 800, 800) (100,)


In [3]:
for i in range(len(pixels)):
    calRays(i,rays,c2w,origins)

In [4]:
vs = voxel_system()
vs.zero_init()

buffer=ti.Vector.field(3,dtype=ti.f32,shape=(800,800))


In [5]:
for i in tqdm(range(100)):
    k=i
    cluster(buffer,origins,rays,k,pixels[k],vs.content)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:18<00:00,  5.34it/s]


In [6]:
def val():
    k=np.random.randint(0,100)
    print('k=', k, end='   ')
    renderBuffer(buffer,vs.content,origins,rays,k)
    img=cv2.flip(cv2.transpose(buffer.to_numpy()[:,:,::-1]*255.), 0).astype(int)
    # print(img)
    cv2.imwrite("./out/2.png", img)    
    
    gt=cv2.flip(cv2.transpose(pixels[k].to_numpy()[:,:,2::-1]*255.), 0).astype(int)
    cv2.imwrite("./out/3.png", gt)
    # print(img.shape, gt.shape)
    print('psnr=', psnr(gt, img), end='   ')
    print('ssim=', ssim(gt, img))

In [7]:
i=0
camera = Camera(vs)
vs.lr[None]=0.003
while camera.gui.running:
    camera.cal_rays()
    camera.render()
    if i<20:
        for j in range(10):
            camera.display()
            k=np.random.randint(0,100)
            renderBuffer(buffer,vs.content,origins,rays,k)
            renderBuffer_back(buffer,vs.content,origins,rays,k,pixels[k],vs.grad)
        vs.update(True)
        vs.zero_grad()
        i+=1
        print(f'epoch {i} fininshed')
        val()
    else:
        camera.display()
    camera.processEvents()
        

epoch 1 fininshed
k= 23   psnr= 14.399477531007474   ssim= 0.7335311368511396
epoch 2 fininshed
k= 58   psnr= 15.037752296975896   ssim= 0.7263728747506649
epoch 3 fininshed
k= 53   psnr= 15.823503990108222   ssim= 0.8103656286592832
epoch 4 fininshed
k= 23   psnr= 17.198911266765794   ssim= 0.7879150911465965


KeyboardInterrupt: 